# NLCD & TCC

## Packages and working directory 

In [ ]:
import os
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

from rasterio.mask import mask
from rasterio.plot import show
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

from shapely.geometry import box
from matplotlib.colors import ListedColormap, BoundaryNorm


# Set the working directory
wd = 'E:/_data/LULC/NLCD_USA/'
os.chdir(wd)

dir_output = r'D:\natcap\invest-mental-health\data\lc_scenarios'

# lc_baseline = os.path.join('nlcd_2019_land_cover_l48_20210604', 'nlcd_2019_land_cover_l48_20210604.img')
lc_baseline = os.path.join('NLCD_Land_Cover_L48_2019424_full_zip', 'NLCD_2011_Land_Cover_L48_20190424.img')
lc_scenario = os.path.join('nlcd_2021_land_cover_l48_20230630', 'nlcd_2021_land_cover_l48_20230630.img')


# print(os.path.basename(lc_baseline))

lc_b_output = os.path.join(dir_output, 'nlcd_2011_land_cover.tif')
lc_s_output = os.path.join(dir_output, 'nlcd_2021_land_cover.tif')



treeCanopy = os.path.join('NLCD_TCC', 'nlcd_tcc_conus_2021_v2021-4.tif')



print(lc_scenario)
print(treeCanopy)

nlcd_2021_land_cover_l48_20230630\nlcd_2021_land_cover_l48_20230630.img
NLCD_TCC\nlcd_tcc_conus_2021_v2021-4.tif


## AOI

In [17]:
# Load the shapefile
dir_shp = 'D:/natcap/invest-mental-health/data'
shapefile_path = os.path.join(dir_shp, 'cb_2019_us_county_500k_06075_clip.shp')
aoi = gpd.read_file(shapefile_path)


# Load the San Francisco boundary (e.g., from a shapefile or GeoJSON)
sf_boundary = gpd.read_file(shapefile_path)

## Load functions

In [18]:
code_dir = r"D:\natcap\invest-mental-health\code"

code_path = os.path.join(code_dir, "function_reproject_and_align.py")

## load function 
with open(code_path) as f:
    exec(f.read())

## NLCD data

### clip lc baseline 

In [21]:
# Ensure the shapefile's CRS matches the raster's CRS
with rasterio.open(lc_baseline) as src_lc:
    nlcd_crs = src_lc.crs
    if sf_boundary.crs != nlcd_crs:
        sf_boundary = sf_boundary.to_crs(nlcd_crs)

# Load the NLCD raster
with rasterio.open(lc_baseline) as src_lc:
    # Clip the raster using the shapefile
    lc_baseline_clipped, clipped_transform = mask(src_lc, sf_boundary.geometry, crop=True)
    
    # Update metadata with the new transform and dimensions
    clipped_meta = src_lc.meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": lc_baseline_clipped.shape[1],
        "width": lc_baseline_clipped.shape[2],
        "transform": clipped_transform
    })

    # Save the clipped raster to a new file
    with rasterio.open(lc_b_output, "w", **clipped_meta) as dest:
        dest.write(lc_baseline_clipped)


### clip lc scenario

In [20]:
# Ensure the shapefile's CRS matches the raster's CRS
with rasterio.open(lc_scenario) as src_lc:
    nlcd_crs = src_lc.crs
    if sf_boundary.crs != nlcd_crs:
        sf_boundary = sf_boundary.to_crs(nlcd_crs)

# Load the NLCD raster
with rasterio.open(lc_scenario) as src_lc:
    # Clip the raster using the shapefile
    lc_scenario_clipped, clipped_transform = mask(src_lc, sf_boundary.geometry, crop=True)
    
    # Update metadata with the new transform and dimensions
    clipped_meta = src_lc.meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": lc_scenario_clipped.shape[1],
        "width": lc_scenario_clipped.shape[2],
        "transform": clipped_transform
    })

    # Save the clipped raster to a new file
    with rasterio.open(lc_s_output, "w", **clipped_meta) as dest:
        dest.write(lc_scenario_clipped)
